In [53]:
import pandas as pd
import numpy as np
import os, sys

# Now you can import the library as usual
import investos as inv
from scipy.stats import ttest_ind
from sklearn.linear_model import LinearRegression
from datetime import datetime
from scipy.stats.mstats import winsorize
import matplotlib.pyplot as plt
import seaborn as sns
from tabulate import tabulate
from sklearn.preprocessing import StandardScaler


In [108]:
# Step 1: loading the dataframe and join them together
# loading
df_loadings = pd.read_parquet('john_rbics_industry.parquet')
df_actual_returns = pd.read_parquet('john_return_prev_1d_open.parquet').rename(columns={"value": "return_1d"})
df_sizes = pd.read_parquet('john_market_cap_open_dil.parquet').rename(columns={"value": "size"})
# log and then winsorize
df_sizes['size'] = np.log(df_sizes['size'])
df_values = pd.read_parquet('john_market_cap_open_dil_to_net_income_ltm.parquet').rename(columns={"value": "value"})
df_loadings = pd.get_dummies(df_loadings, columns=["industry"])

# merge these dataframe into one
df_values = df_values.merge(df_sizes, how='left', on=['id','datetime'])
df_actual_returns = df_actual_returns.merge(df_values, how='left', on=['id','datetime'])
df_loadings = pd.merge(df_actual_returns, df_loadings, how='left', on=None, left_on='id', right_on='id', left_index=False)

for col in df_loadings.columns:
    if col.startswith("industry_"):
        df_loadings[col] = df_loadings[col].astype(int)

In [109]:
df_loadings[(df_loadings.id == 'XR7GZL-R') & (df_loadings.datetime == '2024-01-31')]

,id,datetime,return_1d,value,size,industry_Business Services,industry_Consumer Cyclicals,industry_Consumer Non-Cyclicals,industry_Consumer Services,industry_Energy,industry_Finance,industry_Healthcare,industry_Industrials,industry_Non-Corporate,industry_Non-Energy Materials,industry_Other,industry_Technology,industry_Telecommunications,industry_Utilities
24411632,XR7GZL-R,2024-01-31,0.013623,20.462871,7.755486,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [110]:
# Step 2: Calculate momentum (e.g., 12-month cumulative return)

# sliding window size for volatiltiy and momentum 
#look_back_period_volatility = 252  # Approx. 12 months for daily data
look_back_period_momentum = 30 
#df_loadings['momentum'] = df_loadings.groupby('id')['return_1d'].apply(lambda x: x.rolling(window=look_back_period_momentum).apply(lambda y: np.prod(1 + y) - 1))
#df_loadings['volatility'] = df_loadings.groupby('id')['return_1d'].apply(lambda x: x.rolling(window=look_back_period_volatility).std()).reset_index(level = 'id').rename(columns={"return_1d": "volatility"}).volatility
df_loadings['momentum'] = df_loadings.groupby('id')['return_1d'].rolling(window=look_back_period_momentum).apply(lambda x: np.prod(1 + x) - 1, raw = True).reset_index(level = 'id').rename(columns={"return_1d": "momentum"}).momentum
# drop those nan values less than then window size
df_loadings['country'] = 1

df_loadings.dropna(inplace = True)
df_loadings.replace([np.inf, -np.inf], 0, inplace=True)

In [111]:
df_loadings[(df_loadings.id == 'XR7GZL-R') & (df_loadings.datetime == '2024-01-31')]

,id,datetime,return_1d,value,size,industry_Business Services,industry_Consumer Cyclicals,industry_Consumer Non-Cyclicals,industry_Consumer Services,industry_Energy,...,industry_Healthcare,industry_Industrials,industry_Non-Corporate,industry_Non-Energy Materials,industry_Other,industry_Technology,industry_Telecommunications,industry_Utilities,momentum,country
24411632,XR7GZL-R,2024-01-31,0.013623,20.462871,7.755486,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0.081315,1


In [112]:
# Step 3. Winsorize and standardize

# Identify numeric columns excluding 'id' and 'datetime'
startDate = '2020-01-01'
df_loadings = df_loadings[df_loadings.datetime >= startDate]
cols = [col for col in df_loadings if col not in ['datetime','id','country','return_1d','momentum','period'] and not col.startswith('industry_')]

# Function to winsorize and standardize within each group
def process_group(group):
    for col in cols:
        # Winsorizing numeric columns at 5th and 95th percentiles within the group
        group[col] = winsorize(group[col], limits=[0.05, 0.05])
    
    # Ensure no infinite values are left
    group[cols] = np.nan_to_num(group[cols], nan=0.0, posinf=0.0, neginf=0.0)
    
    # Standardizing numeric columns within the group
    scaler = StandardScaler()
    group[cols] = scaler.fit_transform(group[cols])
    
    return group

# Apply the processing function to each group defined by 'datetime'
df_loadings = df_loadings.groupby('datetime').apply(process_group, include_groups=False)
df_loadings = df_loadings.reset_index().drop(columns=['level_1'])

In [107]:
df_loadings[(df_loadings.id == 'XR7GZL-R') & (df_loadings.datetime == '2024-01-31')]

,datetime,id,return_1d,value,size,industry_Business Services,industry_Consumer Cyclicals,industry_Consumer Non-Cyclicals,industry_Consumer Services,industry_Energy,...,industry_Industrials,industry_Non-Corporate,industry_Non-Energy Materials,industry_Other,industry_Technology,industry_Telecommunications,industry_Utilities,momentum,country,period
3384077,2024-01-31,XR7GZL-R,0.013623,0.281468,-0.007366,0,0,0,0,0,...,0,0,0,0,0,0,0,0.923468,1,2024


In [113]:
#df_loadings

In [116]:
# identify factor effectiveness
# Step 1: calculate t statistics 
def get_t_statistics(model,X,y):
    predictions = model.predict(X)     
    residuals = y - predictions
    # Calculate the residual standard error
    rss = (residuals ** 2).sum()
    n = len(y)      # number of sample
    p = X.shape[1]  # number of factors

    rse = (rss / (n - p - 1)) ** 0.5
    # Calculate t-values for coefficients
    t_values = model.coef_ / rse
    
    #print(model.coef_,rse)
    #print(n,p,rse)
    return t_values

In [117]:

"""
1.
it comes out with an warning like this:
# /var/folders/sg/1nh38dx53_zc3kxcq1cq1f000000gn/T/ipykernel_29543/1917117034.py:11: RuntimeWarning: invalid value encountered in divide
#  t_values = model.coef_ / rse

this is caused by t_values = model.coef_ / rse while certain model.coef_ value is too low (about e-17 level). Overall the code is fine.
We can just mute this warning later

2. some results shows [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] 0.0 which model.coef_ is a zero list, and rse is zero 
as well
"""

factor_returns = {}
# change to freq 2Y to any period you want
df_loadings["period"] = df_loadings["datetime"].dt.to_period(freq = '2Y')

for d in df_loadings["datetime"].unique():
    # Isolating the data for the current date
    df_current = df_loadings[df_loadings["datetime"] == d]

    # outlier modification and factor standarization
    #df_current = get_filter_standarized_df(df_current)
    
    #df_loadings_cp.update(df_current) 
    # identify factor effectiveness
    # Defining the dependent variable (y) and independent variables (X)
    y = df_current["return_1d"]
    #cols = [col for col in df_current.columns if c]
    #X = df_current.drop(columns=["return_1d", "datetime", "id", "period"])
    X = df_current[['country','size','value','momentum']]
    #Performing linear regression
    model = LinearRegression(fit_intercept=False).fit(X, y)

    t_values = get_t_statistics(model,X,y)
    
    # Print t-values for each factor
    # print("T-Values:")
    # for i, factor in enumerate(X.columns):
    #     print(f"{factor}: {t_values[i]}")
    
    # Storing the coefficients and intercept
    factor_returns[d] = {
        "coefficients": model.coef_,
        "intercept": model.intercept_,
        "feature_names": model.feature_names_in_,
        "r2": model.score(X, y),
         # t_values added to the factor_returns dataframe
        "t-values": abs(t_values),
        "period":df_current.period.unique()
    }

/var/folders/sg/1nh38dx53_zc3kxcq1cq1f000000gn/T/ipykernel_54770/1616004176.py:13: RuntimeWarning: invalid value encountered in divide
  t_values = model.coef_ / rse
/var/folders/sg/1nh38dx53_zc3kxcq1cq1f000000gn/T/ipykernel_54770/1616004176.py:13: RuntimeWarning: invalid value encountered in divide
  t_values = model.coef_ / rse
/var/folders/sg/1nh38dx53_zc3kxcq1cq1f000000gn/T/ipykernel_54770/1616004176.py:13: RuntimeWarning: invalid value encountered in divide
  t_values = model.coef_ / rse
/var/folders/sg/1nh38dx53_zc3kxcq1cq1f000000gn/T/ipykernel_54770/1616004176.py:13: RuntimeWarning: invalid value encountered in divide
  t_values = model.coef_ / rse
/var/folders/sg/1nh38dx53_zc3kxcq1cq1f000000gn/T/ipykernel_54770/1616004176.py:13: RuntimeWarning: invalid value encountered in divide
  t_values = model.coef_ / rse
/var/folders/sg/1nh38dx53_zc3kxcq1cq1f000000gn/T/ipykernel_54770/1616004176.py:13: RuntimeWarning: invalid value encountered in divide
  t_values = model.coef_ / rse
/var

In [118]:
factor_returns

{Timestamp('2020-01-01 00:00:00'): {'coefficients': array([0., 0., 0., 0.]),
  'intercept': 0.0,
  'feature_names': array(['country', 'size', 'value', 'momentum'], dtype=object),
  'r2': 1.0,
  't-values': array([nan, nan, nan, nan]),
  'period': <PeriodArray>
  ['2020']
  Length: 1, dtype: period[2Y-DEC]},
 Timestamp('2020-01-02 00:00:00'): {'coefficients': array([ 0.01272042, -0.00198091, -0.00188533,  0.00547472]),
  'intercept': 0.0,
  'feature_names': array(['country', 'size', 'value', 'momentum'], dtype=object),
  'r2': 0.026811783756931606,
  't-values': array([0.48881774, 0.07612194, 0.07244903, 0.21038132]),
  'period': <PeriodArray>
  ['2020']
  Length: 1, dtype: period[2Y-DEC]},
 Timestamp('2020-01-03 00:00:00'): {'coefficients': array([-0.01385337,  0.00073997,  0.000673  ,  0.00188143]),
  'intercept': 0.0,
  'feature_names': array(['country', 'size', 'value', 'momentum'], dtype=object),
  'r2': 0.0026891836050964013,
  't-values': array([0.50630531, 0.02704415, 0.02459633

In [119]:
list_to_insert = [
    [k, factor_returns[k]["r2"], *factor_returns[k]["t-values"],*factor_returns[k]["coefficients"], factor_returns[k]["period"][0]]
    for k in factor_returns    # iterative through the factor returns dict
]
#print(list_to_insert)
#cols = df_loadings.drop(columns=["return_1d", "datetime", "id"]).columns
cols = ['country','size','value','momentum']
# update
cols_with_return = ['returns_' + col for col in cols]
cols_with_t_values = ['t_values_' + col for col in cols]

# df_factor_returns = df.append(pd.Series(list_to_insert, index=['date', 'r2', *cols]), ignore_index=True)  # using append
df_factor_returns = pd.DataFrame(list_to_insert, columns=["datetime", "r2", *cols_with_t_values, *cols_with_return, "period"])


In [122]:
#df_factor_returns

In [121]:
df_factor_returns.groupby('period').mean()

,datetime,r2,t_values_country,t_values_size,t_values_value,t_values_momentum,returns_country,returns_size,returns_value,returns_momentum
period,,,,,,,,,,
2020,2020-07-01 16:45:48.091602944,0.097441,0.333964,0.077674,0.031871,0.871131,0.001341,-0.000199,-0.000145,0.034026
2021,2021-07-02 19:07:35.172413696,0.092173,0.308745,0.076958,0.055203,0.978261,0.000359,-0.000042,0.000092,0.026697
2022,2022-07-02 12:00:00.000000000,0.104989,0.404253,0.068044,0.060445,1.325113,0.002553,-0.004673,-0.001476,-0.015077
2023,2023-07-01 12:00:00.000000000,0.089405,0.281239,0.075662,0.034309,1.027458,0.000737,-0.000201,-0.000066,0.032704
2024,2024-01-15 15:39:07.826086912,0.141387,0.258875,0.092587,0.037981,0.995255,-0.002522,0.001049,0.000448,0.026249


In [120]:
#df_loadings[(df_loadings.id == 'XR7GZL-R') & (df_loadings.datetime == '2024-01-31')]